0. Let's mount our drive

In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Download and install OpenNMT-py 2.0



In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 1.6 MB 12.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [3]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 12.5 MB/s 
     |████████████████████████████████| 16.0 MB 114 kB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


Let's install some libraries:

In [4]:
!pip install sacremoses

     |████████████████████████████████| 895 kB 12.7 MB/s 


In [5]:
MYCLASSFOLDER="SE_personal"
import os
os.environ['PATH'] += ":/content/drive/MyDrive/" + MYCLASSFOLDER + "/scripts"

2. Let's get some data. We can get any parallel data from, e.g. http://opus.nlpl.eu or you can download the ones I will use for this tutorial.

*   English <-> Bulgarian: https://ilk.uvt.nl/~shterion/data/EN-BG.zip
*   English <-> Dutch: https://ilk.uvt.nl/~shterion/data/EN-NL.zip


We should put the data in our drive, otherwise when the session expires we wil lneed to do the same thing again. That is also valid for all models and intermediate processed files.

In [6]:
!MYCLASSFOLDER="SE_personal"
!mkdir /content/drive/MyDrive/$MYCLASSFOLDER/
!ls /content/drive/MyDrive/$MYCLASSFOLDER

mkdir: cannot create directory ‘/content/drive/MyDrive/SE_personal/’: File exists
EN-NL  EN-NL.zip  pred.txt  scripts


In [7]:
cd /content/drive/MyDrive/$MYCLASSFOLDER

/content/drive/MyDrive/SE_personal


In [ ]:
!mkdir scripts 
!wget https://ilk.uvt.nl/~shterion/scripts/train_test_dev.py -O scripts/train_test_dev.py
!wget https://ilk.uvt.nl/~shterion/scripts/bpe.sh -O scripts/bpe.sh

Now let's download scripts for tokenisation and detokanisation:

In [ ]:
!wget https://ilk.uvt.nl/~shterion/scripts/tokenizer.perl -O scripts/tokenizer.perl
!wget https://ilk.uvt.nl/~shterion/scripts/detokenize.perl -O scripts/detokenize.perl
!wget https://ilk.uvt.nl/~shterion/scripts/nonbreaking_prefixes.zip -O scripts/nonbreaking_prefixes.zip
!unzip scripts/nonbreaking_prefixes.zip -d scripts/

In [ ]:
!wget https://ilk.uvt.nl/~shterion/data/EN-NL.zip -O EN-NL.zip
!unzip -K EN-NL.zip

Let's see what we have:

In [ ]:
!head -10 EN-NL/data/data.*

I will continue using the EN-NL, but you can choose whichever you prefer. 

3. Tokenize the data (takes some time)

In [ ]:
!ls -l
!perl scripts/tokenizer.perl -no-escape -l en -threads 16 < EN-NL/data/data.en > EN-NL/data/data.tok.en
!perl scripts/tokenizer.perl -no-escape -l nl -threads 16 < EN-NL/data/data.nl > EN-NL/data/data.tok.nl

total 277116
drwx------ 5 root root      4096 Oct  6 09:19 EN-NL
-rw------- 1 root root 283758586 Dec 11  2020 EN-NL.zip
drwx------ 3 root root      4096 Oct  6 09:28 scripts
Tokenizer Version 1.1
Language: en
Number of threads: 16
Tokenizer Version 1.1
Language: nl
Number of threads: 16


4. Now we need to split the data into train test and development sets. We can do this with the `train_test_dev.py` script provided at: https://ilk.uvt.nl/~shterion/scripts/train_test_dev.py which we downloaded earlier and stored in the `scripts` folder.

In [ ]:
%run scripts/train_test_dev.py --source=EN-NL/data/data.tok.nl --target=EN-NL/data/data.tok.en 

Spliting ...
done.
Cleaning ...
done.
Saving to: /content/drive/MyDrive/SE_personal/EN-NL/data
All done.


Ideally, now that we have all the data split into train test and dev, tokenized and cleaned we would need to perform one more preprocessing - to split it into subword units. However, due to the lack of time we will not do that.

You can do this at home using the provided `bpe.sh` script (https://ilk.uvt.nl/~shterion/scripts/bpe.sh)

5. Now we need to build a vocabulary - a list of words and their numerical representations that will be used by the NNs. We can do this by running the `onmt_build_vocab` using the configuration file provided with the data in the config folder: `config_vocab.yaml`. 

! If you are using BPE make sure that the files you use to build the vocabulary have the correct extension. For simplicity, I have provided the `config_vocab_bpe.yaml` file.

In [ ]:
!onmt_build_vocab -config EN-NL/config/config_vocab.yaml

Let's see what do we have in the vocabulary:

In [ ]:
!wc -l /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg
!head /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg

369354 /content/drive/MyDrive/SE_personal/EN-NL/data/vocab.trg
the	5096581
of	3079463
,	2838613
.	2410655
and	2295364
to	2066867
in	1618990
a	1032684
for	950635
)	801560


In [47]:
import csv
import sys
word_lengths = []
csv.field_size_limit(sys.maxsize)

i = 0

with open("/content/drive/MyDrive/SE_personal/EN-NL/data/vocab.trg") as tsv:
    for line in csv.reader(tsv, delimiter="\t"):
      n_words += 1
      word_lengths.append(len(line[0]))  



average = sum(word_lengths) / len(word_lengths)
print(average)
  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6. Now we can start the training process - takes a loooong time! Be patient! 

In [8]:
!onmt_train -config /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/config/config_train_rnn.yaml -train_from EN-NL/models/model_step_460000.pt

Streaming output truncated to the last 5000 lines.
[2021-10-11 16:23:23,862 INFO] Step 495008/500000; acc:  22.73; ppl: 281.98; xent: 5.64; lr: 0.00001; 140/146 tok/s;   6966 sec
[2021-10-11 16:23:24,121 INFO] Step 495009/500000; acc:  31.58; ppl: 37.92; xent: 3.64; lr: 0.00001; 147/147 tok/s;   6966 sec
[2021-10-11 16:23:24,319 INFO] Step 495010/500000; acc:  33.33; ppl: 40.24; xent: 3.69; lr: 0.00001; 126/137 tok/s;   6966 sec
[2021-10-11 16:23:24,491 INFO] Step 495011/500000; acc:  31.58; ppl: 109.33; xent: 4.69; lr: 0.00001; 152/111 tok/s;   6966 sec
[2021-10-11 16:23:24,740 INFO] Step 495012/500000; acc:  35.00; ppl: 35.80; xent: 3.58; lr: 0.00001; 157/161 tok/s;   6967 sec
[2021-10-11 16:23:24,980 INFO] Step 495013/500000; acc:  23.53; ppl: 76.88; xent: 4.34; lr: 0.00001; 121/142 tok/s;   6967 sec
[2021-10-11 16:23:25,117 INFO] Step 495014/500000; acc:  45.45; ppl: 101.59; xent: 4.62; lr: 0.00001;  73/ 80 tok/s;   6967 sec
[2021-10-11 16:23:25,407 INFO] Step 495015/500000; acc:  

7. Now that the training is over we can translate using the model. 
You can use the `config_trans.yaml` file provided in the config folder. 
It will translate the test set with the first saved model. Change these to any arbitrary file you wish to translate and to any model you would like to use.

In [10]:
!onmt_translate -config /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/config/config_trans.yaml

Streaming output truncated to the last 5000 lines.
PRED SCORE: -33.0476
GOLD 287: At its 455th plenary session , held on 15 and 16 July 2009 ( meeting of 16 July ) , the European Economic and Social Committee adopted the following opinion by 114 votes to 1 vote against .
GOLD SCORE: -70.6790

[2021-10-11 17:10:26,116 INFO] 
SENT 288: ['Het', 'door', 'de', 'Commissie', 'ingestelde', 'onderzoek', 'wees', 'uit', 'dat', 'er', 'in', 'geen', 'enkel', 'EER-land', 'waar', 'zowel', 'E.ON', 'als', 'Powergen', 'aanwezig', 'zijn', ',', 'sprake', 'is', 'van', 'een', 'overlapping', 'van', 'de', 'activiteiten', 'van', 'de', 'partijen', ',', 'met', 'uitzondering', 'van', 'de', 'handel', 'in', 'elektriciteit', 'in', 'de', 'Noordse', 'markten', 'en', 'Nederland', '.']
PRED 288: The Commission ’ s report on the basis of a number of products in order to ensure that it is necessary for the basis of a number of information , and in the light of a " " " " " " in the European Union .
PRED SCORE: -86.6821
GOLD